In [1]:
import os
import sys

os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable
os.environ["PYSPARK_PYTHON"]        = sys.executable



In [2]:
from pyspark import SparkContext
from sympy import *
from drudge import *
from gristmill import *
 

In [3]:
import re

# ----------------------------------------------------------------------------
# 1) Start Spark & BCS quasiparticle drudge
# ----------------------------------------------------------------------------
ctx = SparkContext('local[2]', 'bcs_ccsd')
dr  = ReducedBCSDrudge(ctx)
dr.full_simplify = True
print("finished")

25/11/20 17:18:42 WARN Utils: Your hostname, Swarnamoys-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 168.5.60.33 instead (on interface en0)
25/11/20 17:18:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/20 17:18:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
                                                                                

finished


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 51568)
Traceback (most recent call last):
  File "/Users/swarnamoyghosh/anaconda3/envs/drudge_fix/lib/python3.9/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/swarnamoyghosh/anaconda3/envs/drudge_fix/lib/python3.9/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Users/swarnamoyghosh/anaconda3/envs/drudge_fix/lib/python3.9/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/swarnamoyghosh/anaconda3/envs/drudge_fix/lib/python3.9/socketserver.py", line 747, in __init__
    self.handle()
  File "/Users/swarnamoyghosh/anaconda3/envs/drudge_fix/lib/python3.9/site-packages/pyspark/accumulators.py", line 281, in handle
    poll(accum_updates)
  File "/Users/swarnamoyghosh/anaconda3/envs/drudge

In [4]:
P, Pdag, N = dr.names.P, dr.names.Pdag, dr.names.N

In [5]:
p,q,r,s,i,j,k,l = dr.names.A_dumms[:8]
 
 
u, v = IndexedBase('u'), IndexedBase('v')
E0 = Symbol(r'G')
H120, H021 = IndexedBase('h120') , IndexedBase('h021')
H011, H110 = IndexedBase('h011'), IndexedBase('h110')
H100, H011 = IndexedBase('h100'), IndexedBase('h011')
H001, H020 = IndexedBase('h001'), IndexedBase('h020')
H010 = IndexedBase('h010') 
H000 = IndexedBase('h000')

#L = Range('L', 1, Lmax)
#dr.set_dumms(L, (p, q, r, s))
#dr.add_resolver_for_dumms()


In [6]:
#Lag = dr.sum((p,L), E0*u[p]**2*v[p])
#Lag = 2*E0*u[p]**2*v[p]*v[q]*u[q]  + v[p]**2*u[p]*u[q] + 2*u[p]**2*v[p]*v[q]*u[q]
Lag =  2*u[p]**2*v[p]*v[q]
Lag = dr.einst(Lag)
#print(Lag.free_inds)   # free index *objects*
#print(Lag.free_vars)   # free scalar symbols


In [7]:
Lag.display()

<IPython.core.display.Math object>

In [8]:
dlag_dt = Lag.diff(v[p])
#Dlag_Dt = diff(Lag,u[p])
print('differentiation')

differentiation


In [9]:
dlag_dt.display()

<IPython.core.display.Math object>

In [10]:
H11, H20 , H02 = IndexedBase('H11'), IndexedBase('H20'), IndexedBase('H02')
H04, H40 = IndexedBase('H04'), IndexedBase('H40')
H22, Hb22 = IndexedBase('H22'), IndexedBase('Hb22')
H31, H13 = IndexedBase('H31'), IndexedBase('H13')

In [11]:
expr = H11[p]*N[p] + H02[p]*Pdag[p] +H20[p]*P[p]+H22[p,q]*N[p]*N[q]+Hb22[p,q]*Pdag[p]*P[q]\
    +H40[p,q]*P[p]*P[q]+H04[p,q]*Pdag[p]*Pdag[q]+H13[p,q]*Pdag[p]*N[q]+H31[p,q]*N[p]*P[q]
H_N = dr.einst(expr).simplify().merge()

In [12]:
H_N.display()

<IPython.core.display.Math object>

In [13]:
t1, t2, t3 = IndexedBase('t1'), IndexedBase('t2'), IndexedBase('t3')

z1, z2 = IndexedBase('z1'), IndexedBase('z2')

Z1 = dr.einst(z1[p]*P[p])
Z2 = dr.einst(Rational(1,2)*z2[p,q]*P[p]*P[q])

T1 = dr.einst(t1[p] * Pdag[p])
T2 = dr.einst(Rational(1, 2) * t2[p,q] *Pdag[p]*Pdag[q])
T3 = dr.einst(Rational(1,6)*t3[p,q,r]*Pdag[p]*Pdag[q]*Pdag[r])

Z = Z1 + Z2
cluster = T1 + T2 #+ T3
Z.display()
#cluster.display()

<IPython.core.display.Math object>

In [14]:
dr.set_symm(t2, Perm([1, 0],IDENT), valence=2) 
dr.set_symm(z2, Perm([1, 0],IDENT), valence=2) 
dr.set_symm(H04,Perm([1,0],IDENT))
dr.set_symm(H40,Perm([1,0],IDENT)) 
#dr.set_symm(t3, Perm([1,0,2],IDENT),Perm([2,1,0],IDENT), Perm([0,2,1],IDENT), valence=3)
print("Perm works!")

Perm works!


In [15]:
zero_term = [(H40[p,p],0),(H04[p,p],0),(t2[p,p],0),(z2[p,p],0)]

In [16]:
Hbar = H_N
curr = H_N
fact = 1
for n in range(4):                              # up to 4-fold for 2-body H
    comm = (curr | cluster).simplify()          # ad_T^{n+1}(H_N)
    fact *= (n + 1)                             # 1!,2!,3!,4!
    Hbar += comm / fact                         # add 1/(n+1)! term
    curr = comm    
print("Hbar evaluation done")

[Stage 122:>                                                        (0 + 2) / 2]

Hbar evaluation done


In [ ]:
E_eqn  = Hbar.simplify().eval_vev().simplify()
E_eqn = E_eqn.subst_all(zero_term)
#print(E_eqn.latex())
E_eqn.display()

In [ ]:
s1_eqn_CCSD = (P[p] * Hbar).simplify().eval_vev().simplify()
print('evaluation done')

In [ ]:
s1_eqn_CCSD = s1_eqn_CCSD.subst_all(zero_term)
print(s1_eqn_CCSD.latex())

In [ ]:
s2_eqn_CCSD = (P[p]*P[q] * Hbar).simplify().eval_vev().simplify()
s2_eqn_CCSD = s2_eqn_CCSD.subst_all(zero_term)
print('evaluation done')

In [ ]:
print(s2_eqn_CCSD.latex())

In [22]:
def drop_delta_terms_str(tens: Tensor) -> Tensor:
    # Remove any term whose printed form contains 'KroneckerDelta('
    filt = tens.terms.filter(lambda term: 'KroneckerDelta' not in str(term))
    return Tensor(tens._drudge, filt).merge().simplify()

s1_eqn_CCSD = drop_delta_terms_str(s1_eqn_CCSD)
s2_eqn_CCSD = drop_delta_terms_str(s2_eqn_CCSD)

NameError: name 's1_eqn_CCSD' is not defined

In [ ]:
print(s1_eqn_CCSD.latex())

In [ ]:
print(s2_eqn_CCSD.latex())

In [ ]:
'''Defining CC Lagrangian
L = <0|(1+Z) bar{H}|0> 
'''
'''Z1,Z2 = IndexedBase('z1'), IndexedBase('z2')
R1,R2 = IndexedBase('R1'), IndexedBase('R2')
 
expr1 = dr.define(R1[p],s1_eqn_CCSD)
 
expr2 = dr.define(R2[p,q],s2_eqn_CCSD)
L =  E_eqn + Z1[p]*s1_eqn_CCSD + Z2[p,q]*s2_eqn_CCSD
L = dr.einst(L).merge()'''

In [ ]:
print(L.latex())

In [ ]:
dL_dt_1 = L.diff(t1[p]).simplify().merge()

#dL_dt = L.diff(t1[r])


In [ ]:
print(dL_dt_1.latex())

In [ ]:
dL_dt_2 = L.diff(t2[r,s]).simplify().merge()

In [ ]:
print(dL_dt_2.latex())

In [17]:
'''Defining CC Lagrangian
L = <0|(1+Z) bar{H}|0> and dL/dt = 0 is the Z equations. dL/dt = <0|(1+Z)[bar{H},Q_{mu}]|0>
'''

#dL_dt_1 = (1+Z)*(Hbar|Pdag[p]).simplify().eval_vev().simplify().merge()
dL_dt_1 = ((1+Z)*(Hbar|Pdag[p])).simplify().eval_vev().simplify()

In [18]:
dL_dt_1 = dL_dt_1.subst_all(zero_term)
print(dL_dt_1.latex())

2 H^{(11)}_{p}  z^{(1)}_{p}   + 4 z^{(1)}_{p}  H^{(22)}_{p,p}  - 4 t^{(1)}_{p}  z^{(1)}_{p}  H^{(31)}_{p,p}  - 2 H^{(20)}_{p}  t^{(1)}_{p}  z^{(1)}_{p}   + H^{(20)}_{p}   + \sum_{q \in A} z^{(1)}_{q}  Hb^{(22)}_{q,p}   + \sum_{q \in A} 2 t^{(1)}_{q}  H^{(40)}_{q,p}   + \sum_{q \in A} 2 H^{(13)}_{q,p}  z^{(2)}_{q,p}  - \sum_{q \in A} 4 z^{(1)}_{p}  H^{(40)}_{q,p}  t^{(2)}_{q,p}  - \sum_{q \in A} 4 H^{(31)}_{p,p}  t^{(2)}_{q,p}  z^{(2)}_{q,p}  - \sum_{q \in A} 4 H^{(31)}_{q,p}  t^{(2)}_{q,p}  z^{(2)}_{q,p}  - \sum_{q \in A} 2 t^{(1)}_{q}^{2}  z^{(1)}_{q}  H^{(40)}_{q,p}  - \sum_{q \in A} 2 t^{(1)}_{q}^{2}  H^{(31)}_{p,q}  z^{(2)}_{q,p}  - \sum_{q \in A} 2 H^{(20)}_{p}  t^{(2)}_{q,p}  z^{(2)}_{q,p}  - \sum_{q \in A} 2 t^{(1)}_{p}  Hb^{(22)}_{q,p}  z^{(2)}_{q,p}   + \sum_{q \in A} 2 t^{(1)}_{q}  z^{(1)}_{p}  H^{(31)}_{p,q}   + \sum_{q \in A} 2 t^{(1)}_{q}  z^{(1)}_{q}  H^{(31)}_{q,p}   + \sum_{q \in A} 4 t^{(1)}_{q}  H^{(22)}_{p,q}  z^{(2)}_{q,p}   + \sum_{q \in A} 4 t^{(1)}_{q}  H^{(22)}_

In [26]:
dL_dt_2 = ((1+Z)*(Hbar|Pdag[p]*Pdag[q])).simplify().eval_vev().simplify()
print('finish')

[Stage 2093:>                                                       (0 + 2) / 2]

finish


In [27]:
dL_dt_2 = dL_dt_2.subst_all(zero_term)
print(dL_dt_2.latex())

2 H^{(40)}_{p,q}   + H^{(20)}_{p}  z^{(1)}_{q}   + H^{(20)}_{q}  z^{(1)}_{p}   + 2 H^{(11)}_{p}  z^{(2)}_{p,q}   + 2 H^{(11)}_{q}  z^{(2)}_{p,q}   + 2 z^{(1)}_{p}  H^{(31)}_{p,q}   + 2 z^{(1)}_{q}  H^{(31)}_{q,p}   + 4 H^{(22)}_{p,p}  z^{(2)}_{p,q}   + 4 H^{(22)}_{p,q}  z^{(2)}_{p,q}   + 4 H^{(22)}_{q,p}  z^{(2)}_{p,q}   + 4 H^{(22)}_{q,q}  z^{(2)}_{p,q}  - 4 t^{(1)}_{p}  z^{(1)}_{p}  H^{(40)}_{p,q}  - 4 t^{(1)}_{p}  H^{(31)}_{p,p}  z^{(2)}_{p,q}  - 4 t^{(1)}_{p}  H^{(31)}_{q,p}  z^{(2)}_{p,q}  - 4 t^{(1)}_{q}  z^{(1)}_{q}  H^{(40)}_{p,q}  - 4 t^{(1)}_{q}  H^{(31)}_{p,q}  z^{(2)}_{p,q}  - 4 t^{(1)}_{q}  H^{(31)}_{q,q}  z^{(2)}_{p,q}  -  4 \delta_{p q} z^{(1)}_{p}  H^{(31)}_{p,p}  - 2 H^{(20)}_{p}  t^{(1)}_{p}  z^{(2)}_{p,q}  -  2 \delta_{p q} H^{(20)}_{p}  z^{(1)}_{p}  - 2 H^{(20)}_{q}  t^{(1)}_{q}  z^{(2)}_{p,q}   + 8 H^{(40)}_{p,q}  t^{(2)}_{p,q}  z^{(2)}_{p,q}   + 8 t^{(1)}_{p}  t^{(1)}_{q}  H^{(40)}_{p,q}  z^{(2)}_{p,q}   + \sum_{r \in A} Hb^{(22)}_{r,p}  z^{(2)}_{r,q}   + \sum_{r 

In [28]:
dL_dt_2 = drop_delta_terms_str(dL_dt_2)

In [29]:
L1 = IndexedBase('L1')
L2 = IndexedBase('L2')

expr1 = dr.define(L1[p],dL_dt_1)
 
 
Res2 = IndexedBase('Res2')
expr2 = dr.define(L2[p,q],dL_dt_2)
 

eval_equ0 = optimize(
    [expr1,expr2],
    interm_fmt='tau{}',drop_cutoff=2
)
print("Original cost:", get_flop_cost(eval_equ0))

fort = FortranPrinter()
print("Optimized cost:", get_flop_cost(eval_equ0))
 
evals = fort.doprint(eval_equ0)
with open('BCS_CCSD_Z.f90', 'w') as fp:
    fp.write(evals)

print("Printed!")

Original cost: 6*na**3 + 63*na**2 + 38*na
Optimized cost: 6*na**3 + 63*na**2 + 38*na
Printed!
